In [7]:
import glob
import numpy as np
import scipy as sp
import einops as eo
import matplotlib.pyplot as plt
import math
import plotly.graph_objects as go
from scipy.spatial.transform import Rotation
import magpylib as magpy
import os



M0 = 1480 #mT
shape = [3*25.4/16, 25.4/8] #radius and height
x0 = np.array([0,0,10, 0,0,1])

In [8]:
def B_dipole(position, rotation, M0, shape):
    R = np.sqrt(np.sum(position**2, axis=1))
    B = (M0 * (shape[0]) ** 2 * shape[1] / (16)) * (
        (
            3
            * position
            / R[:, np.newaxis] ** 5
            * (eo.einsum(position, rotation, "sensor dim,  dim -> sensor"))[
                :, np.newaxis
            ]
        )
        - rotation[np.newaxis, :] / (R[:, np.newaxis] ** 3)
    )
    return B


def getField_dipole(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def getField_dipole_fixed(x, positions, M0, shape):
    position = x[:3]
    axis = x[3:]
    return B_dipole(positions - position, axis, M0, shape)


def cost_dipole(x, B, positions, M0, shape):
    diff = getField_dipole(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)


def getField_cylinder(x, positions, M0, shape):
    B=magpy.getB(
        sources="Cylinder",
        position=x[:3],
        orientation=Rotation.align_vectors(x[3:], np.array([0, 0, 1]))[0],
        observers=positions,
        dimension=shape,
        polarization=(0, 0, M0),
    )
    return B

def cost_cylinder(x, B, positions, M0, shape):
    diff = getField_cylinder(x, positions, M0, shape) - B
    return np.sum((diff) ** 2)

def minimize(x0, B, positions, M0, shape, *args):
    #print("Starting mimimization")
    b_args = (B, positions, M0, shape)
    cons = [{"type": "eq", "fun": lambda x: x[3] ** 2 + x[4] ** 2 + x[5] ** 2 - 1}]
    bounds = [(-100, 100), (-100, 100), (0, 100), (-1, 1), (-1, 1), (-1, 1)]
    res = sp.optimize.minimize(
        fun=cost_dipole, x0=x0, args=b_args, tol=1e-100, constraints=cons, bounds=bounds, *args
    ).x  
    #print(f"Finished mimimization with shape {b_args[3]} at {res}")
    return res

def circle_radius(x1, y1, x2, y2, x3, y3):
    # Calculate lengths of sides of the triangle formed by the three points
    a = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    b = math.sqrt((x3 - x2) ** 2 + (y3 - y2) ** 2)
    c = math.sqrt((x3 - x1) ** 2 + (y3 - y1) ** 2)

    # Calculate the semi-perimeter of the triangle
    s = (a + b + c) / 2

    # Calculate the area of the triangle using Heron's formula
    area = math.sqrt(s * (s - a) * (s - b) * (s - c))

    # Calculate the radius of the circle using the formula: radius = (abc) / (4 * area)
    radius = (a * b * c) / (4 * area)
    
    return radius

def comparison_plot(locact, locpred, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    # Plotting
    plt.figure(figsize=(8, 6))
    plt.scatter(index, pred, label='Predicted' + rel)
    plt.scatter(index, act, label='Actual' + rel)
    plt.xlabel('Index')
    plt.ylabel(axis + '-component')
    plt.title('Comparison of '+axis+ ' Location' + rel+ ': Predicted vs Actual')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def comp_plot3D(locact, locpred, xact, yact, axis, relative):

    if axis == 'x':
        ax = 0
    elif axis == 'y':
        ax = 1
    else:
        ax = 2

    index = list(range(len(locact[:,ax])))
    act = locact[:, ax]
    pred = locpred[:, ax]

    rel = ''

    if relative==True:
        act = [x - locact[0, ax] for x in locact[:, ax]]
        pred = [x - locpred[0, ax] for x in locpred[:, ax]]
        rel = ' (Relative)'

    fig = go.Figure(data=[
        go.Scatter3d(x=xact, y=yact, z=pred, name='Predicted' + rel),
        go.Scatter3d(x=xact, y=yact, z=act, name='Actual' + rel)])
    fig.show()

    return

def displacement_plot(actual, predicted):

    dispx = np.array(predicted[:,0]-predicted[:,0][0])
    dispy = np.array(predicted[:,1]-predicted[:,1][0])
    dispz = np.array(predicted[:,2]-predicted[:,2][0])
    disp = np.sqrt(dispx**2+dispy**2+dispz**2)
    actdispx = np.array(actual[:,0]-actual[:,0][0])
    actdispy = np.array(actual[:,1]-actual[:,1][0])
    actdispz = np.array(actual[:,2]-actual[:,2][0])
    actdisp = np.sqrt(actdispx**2+actdispy**2+actdispz**2)

    plt.figure(figsize=(8, 6))
    plt.scatter(list(range(len(disp))), disp, label='Predicted Displacement')
    plt.scatter(list(range(len(actdisp))), actdisp, label='Actual Displacement')
    plt.xlabel('Index')
    plt.ylabel('Displacement (mm)')
    plt.title('Magnet Displacement')
    plt.legend()
    plt.grid(True)
    plt.show()

    return

def importfitting(folder):

    file_list = sorted(glob.glob(f'{folder}/*.npz'))
    print(len(file_list))
    data = np.load(file_list[0])
    pos = data['pos'][0]

    locpred = [np.array([0,0,0])]
    anglepred = [np.array([0,0,0])]
    Bstdev = [np.array(pos)]
    for x in range(len(file_list)):
        data = np.load(file_list[x])
        mags = data['mags']
        pos = data['pos'][0]
        B = np.mean(mags, axis=0)
        mags_stdev = np.std(mags, axis=0)
        x_res = minimize(x0, B, pos, M0, shape)
        locpred = np.append(locpred, [x_res[0:3]], axis=0)
        anglepred = np.append(anglepred, [x_res[3:6]], axis=0)
        Bstdev = np.append(Bstdev, [mags_stdev], axis=0)
    locpred = np.delete(locpred, 0, axis=0)
    anglepred = np.delete(anglepred, 0, axis=0)
    Bstdev = np.delete(Bstdev, 0 ,axis=0)

    return pos, mags, locpred, anglepred, Bstdev

def actualangle(xind, yind, xscal, xcon, yscal, ycon, isoff):

    angact = [np.array([0, 0, 0])]
    for y in range(yind):
        for x in range(xind):
            R = Rotation.from_euler('ZYX' ,np.array([0, yscal*y-ycon, xscal*x-xcon]), degrees=True)
            norm = np.array([[0, 0, 1]])
            rotated = R.apply(norm, inverse=False)[0]
            #add = np.array([np.sin(np.deg2rad(2*y-4)), -np.sin(np.deg2rad(2*x-4)), np.cos(np.deg2rad(2*y-4))*np.cos(np.deg2rad(2*x-4))])
            angact = np.append(angact, [rotated], axis=0)
    angact = np.delete(angact, 0, axis=0)
    locact = angact*isoff

    return angact, locact

In [9]:
folder = os.path.join('..', './MLX080142024_D32N52_ServerComp')
pos, mags, locpred, anglepred, Bstdev = importfitting(folder)
np.append(locpred, anglepred, axis=1)

5


array([[ 5.06844029,  4.29904962, 12.88929415,  0.06877107,  0.13492092,
        -0.98846694],
       [ 5.06858017,  4.29890413, 12.889188  ,  0.06883553,  0.13490748,
        -0.98846428],
       [ 5.06738874,  4.29965806, 12.88960038,  0.06898349,  0.13483551,
        -0.98846379],
       [ 5.06882859,  4.29934801, 12.88939233,  0.06881072,  0.1348744 ,
        -0.98847053],
       [ 5.06907723,  4.29859807, 12.88938489,  0.06868944,  0.13496897,
        -0.98846605]])

In [10]:
file_list = sorted(glob.glob(f'{folder}/*.npz'))
len(file_list)
data = np.load(file_list[0])
pos = data['pos'][0]

all_mags = [np.array([0,0,0])]
for x in range(len(file_list)):
    data = np.load(file_list[x])
    mags = data['mags']
    all_mags = np.append(all_mags, mags[0], axis=0)
    pos = data['pos'][0]
    B = np.mean(mags, axis=0)
    mags_stdev = np.std(mags, axis=0)
all_mags = np.delete(all_mags, 0, axis=0)

print(all_mags)
bx = all_mags[:,0]
by = all_mags[:,1]
bz = all_mags[:,2]

print(bx)
len(bx)

[[ 0.1986   -1.8501   -0.609356]
 [-0.4371   -1.7115   -0.55902 ]
 [-0.7497   -1.2534   -0.217316]
 [-0.7017   -0.6822    0.051788]
 [ 0.5514   -2.6382   -2.422904]
 [-1.0122   -2.3916   -2.2022  ]
 [-1.5828   -1.6077   -1.025112]
 [-1.1754   -0.7311   -0.213928]
 [ 0.9987   -1.6722   -5.418864]
 [-1.8588   -1.5858   -4.670116]
 [-2.424    -0.8529   -2.033284]
 [-1.6224   -0.405    -0.460284]
 [ 1.0161    1.326    -5.503564]
 [-1.8516    1.2507   -4.974068]
 [-2.481     0.6075   -2.178   ]
 [-1.6617    0.2373   -0.551276]
 [ 0.201    -1.8492   -0.610324]
 [-0.438    -1.7097   -0.559988]
 [-0.7512   -1.2543   -0.217316]
 [-0.7014   -0.6822    0.049852]
 [ 0.5514   -2.6385   -2.425808]
 [-1.0131   -2.3904   -2.201716]
 [-1.5825   -1.6095   -1.027532]
 [-1.1757   -0.732    -0.211508]
 [ 1.0002   -1.6722   -5.419832]
 [-1.8603   -1.5876   -4.667212]
 [-2.4234   -0.8523   -2.03522 ]
 [-1.6236   -0.4047   -0.463188]
 [ 1.0161    1.326    -5.503564]
 [-1.8498    1.2516   -4.969228]
 [-2.4813 

80

In [29]:
bx7 = []
by7 = []
bz7 = []
for x in range(len(bx)):
    if x % 16 == 6:
        bx7.append(bx[x])
        by7.append(by[x])
        bz7.append(bz[x])
print(bx7)
print(pos)
print(locpred)

[-1.5828, -1.5825, -1.5828, -1.581, -1.5822]
[[ 6.75 -6.75  0.  ]
 [ 2.25 -6.75  0.  ]
 [-2.25 -6.75  0.  ]
 [-6.75 -6.75  0.  ]
 [ 6.75 -2.25  0.  ]
 [ 2.25 -2.25  0.  ]
 [-2.25 -2.25  0.  ]
 [-6.75 -2.25  0.  ]
 [ 6.75  2.25  0.  ]
 [ 2.25  2.25  0.  ]
 [-2.25  2.25  0.  ]
 [-6.75  2.25  0.  ]
 [ 6.75  6.75  0.  ]
 [ 2.25  6.75  0.  ]
 [-2.25  6.75  0.  ]
 [-6.75  6.75  0.  ]]
[[ 5.06844029  4.29904962 12.88929415]
 [ 5.06858017  4.29890413 12.889188  ]
 [ 5.06738874  4.29965806 12.88960038]
 [ 5.06882859  4.29934801 12.88939233]
 [ 5.06907723  4.29859807 12.88938489]]


In [28]:
serverloc = np.array([[5.06388, 4.29981, 12.88836],
                     [5.06915, 4.29835, 12.88869],
                     [5.06589, 4.29664, 12.88889],
                     [5.06819, 4.30095, 12.89004],
                     [5.07075, 4.29614, 12.88891]])
serverang = np.array([[0.06970, 0.13458, -0.98845],
                     [0.06877, 0.13519, -0.98843],
                     [0.06927, 0.13516, -0.98840],
                     [0.06892, 0.13432, -0.98854],
                     [0.06838, 0.13528, -0.98844]])
print(anglepred)

[[ 0.06877107  0.13492092 -0.98846694]
 [ 0.06883553  0.13490748 -0.98846428]
 [ 0.06898349  0.13483551 -0.98846379]
 [ 0.06881072  0.1348744  -0.98847053]
 [ 0.06868944  0.13496897 -0.98846605]]


In [12]:
index = np.array(range(len(bx7)))
measuredbx = go.Scatter(x=index, y=bx7, mode='markers')
layoutbx = go.Layout(title='b_x over time')
figbx = go.Figure(data=[measuredbx], layout=layoutbx)
figbx.update_yaxes(scaleanchor='y')
figbx.update_layout(
    xaxis_title="Index",
    yaxis_title="b_x [mT]"
)
figbx.show()

In [13]:
measuredby = go.Scatter(x=index, y=by7, mode='markers')
layoutby = go.Layout(title='b_y over time')
figby = go.Figure(data=[measuredby], layout=layoutby)
figby.update_yaxes(scaleanchor='y')
figby.update_layout(
    xaxis_title="Index",
    yaxis_title="b_y [mT]"
)
figby.show()

In [14]:
measuredbz = go.Scatter(x=index, y=bz7, mode='markers')
layoutbz = go.Layout(title='b_z over time')
figbz = go.Figure(data=[measuredbz], layout=layoutbz)
figbz.update_yaxes(scaleanchor='y')
figbz.update_layout(
    xaxis_title="Index",
    yaxis_title="b_z [mT]"
)
figbz.show()

In [15]:
size = 2

measured = go.Cone(x=locpred[:,0], y=locpred[:,1], z=locpred[:,2], u=anglepred[:,0], v=anglepred[:,1], w=anglepred[:,2], 
                             sizemode="absolute", sizeref=size, )

#calculated = go.Cone(x=locact[:,0], y=locact[:,1], z=locact[:,2], u=actang[:,0], v=actang[:,1], w=actang[:,2], 
#                             sizemode="absolute", sizeref=size, colorscale=[(0, 'blue'), (1, 'blue')])

layout = go.Layout(title='Location Prediction', scene=dict(aspectmode='data', xaxis_title='x [mm]', yaxis_title='y [mm]', zaxis_title='z [mm]'))

fig = go.Figure(data=[measured], layout=layout)

fig.show()

In [16]:
a = 0
b = 1

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=locpred[:,2], mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Index"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='Rotation over Time', scene=dict(xaxis_title='x [mm]', yaxis_title='y [mm]', zaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [17]:
a = 0
b = 2

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=np.array(range(len(locpred[:,a]))), mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Colorbar"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='x vs z', scene=dict(xaxis_title='x [mm]', yaxis_title='z [mm]', zaxis_title='Index'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [18]:
a = 1
b = 2

measured2D = go.Scatter3d(x=locpred[:,a], y=locpred[:,b], z=np.array(range(len(locpred[:,a]))), mode='markers', marker=dict(
        size=12,
        color=np.array(range(len(locpred[:,a]))),                # set color to an array/list of desired values
        colorbar=dict(
            title="Index"
        ),
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    ))
#calc2D = go.Scatter(x=locact[:,a], y=locact[:,b], mode='markers')
layout = go.Layout(title='y vs z', scene=dict(xaxis_title='y [mm]', yaxis_title='z [mm]', zaxis_title='Index'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

In [19]:
a = 0
b = 1

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='x vs y (measured)')
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="y [mm]"
)
fig2.update_yaxes(scaleanchor='x')
fig2.show()

print(locpred)
print(anglepred)

[[ 5.06844029  4.29904962 12.88929415]
 [ 5.06858017  4.29890413 12.889188  ]
 [ 5.06738874  4.29965806 12.88960038]
 [ 5.06882859  4.29934801 12.88939233]
 [ 5.06907723  4.29859807 12.88938489]]
[[ 0.06877107  0.13492092 -0.98846694]
 [ 0.06883553  0.13490748 -0.98846428]
 [ 0.06898349  0.13483551 -0.98846379]
 [ 0.06881072  0.1348744  -0.98847053]
 [ 0.06868944  0.13496897 -0.98846605]]


In [20]:
a = 0
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='x vs z (measured)', scene=dict(xaxis_title='x [mm]', yaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.update_layout(
    xaxis_title="x [mm]",
    yaxis_title="z [mm]"
)
fig2.show()

In [21]:
a = 1
b = 2

measured2D = go.Scatter(x=locpred[:,a], y=locpred[:,b], mode='markers')
layout = go.Layout(title='y vs z (measured)', scene=dict(xaxis_title='y [mm]', yaxis_title='z [mm]'))
fig2 = go.Figure(data=[measured2D], layout=layout)
fig2.update_yaxes(scaleanchor='x')
fig2.update_layout(
    xaxis_title="y [mm]",
    yaxis_title="z [mm]"
)
fig2.show()


Plot bx and by for 06242024_rotlog and compare to Satwik's bfield plot (try fitting with data from fewer sensors - center 4 maybe)
take rotation at different radii
15mm between sensors and magnet, 5mm radius about Z axis.


Check why rerunning minimzation doesn't result in different predicted location values.